# PII Detection Experiments with Hugging Face Dataset

This notebook explores the [ai4privacy/pii-masking-200k](https://huggingface.co/datasets/ai4privacy/pii-masking-200k) dataset and tests OpenAI's ability to detect PII.

## Setup and Imports

In [3]:
# Install required packages if needed
# !pip install datasets openai python-dotenv

In [3]:
import os
import random
from pathlib import Path
from datasets import load_dataset
from openai import OpenAI
import json
from dotenv import load_dotenv

# Load environment variables from .env file in parent directory (project root)
env_file = Path(__file__).parent.parent / '.env' if '__file__' in globals() else Path.cwd().parent / '.env'

print(f"Looking for .env file at: {env_file}")
if env_file.exists():
    print(f"✓ Found .env file")
    load_dotenv(dotenv_path=env_file)
else:
    print(f"⚠️  .env file not found at {env_file}")
    print("   Will try loading from default locations")
    load_dotenv()

Looking for .env file at: /Users/kodlan/workspaces/python/pii-llm/.env
✓ Found .env file


## Load the Dataset

Loading the PII masking dataset from Hugging Face.

In [4]:
print("Loading dataset...")

# Get Hugging Face token from environment for faster downloads (optional)
hf_token = os.getenv("HF_TOKEN")

if hf_token:
    # Check if it's a placeholder value
    if hf_token.startswith("your-") or "placeholder" in hf_token.lower() or "example" in hf_token.lower():
        print("⚠️  HF_TOKEN appears to be a placeholder value - will use unauthenticated access")
        print("   Edit your .env file and add your actual Hugging Face token for faster downloads")
        hf_token = None
    else:
        print("✓ Using HF_TOKEN for authenticated access (faster downloads)")
else:
    print("⚠️  No HF_TOKEN found - using unauthenticated access (slower)")
    print("   Get a token at https://huggingface.co/settings/tokens to speed up downloads")

# Load dataset with token if available
dataset = load_dataset("ai4privacy/pii-masking-200k", token=hf_token if hf_token else None)
train_data = dataset["train"]

print(f"\nDataset loaded successfully!")
print(f"Total number of samples: {len(train_data)}")
print(f"\nColumn names: {train_data.column_names}")

Loading dataset...
✓ Using HF_TOKEN for authenticated access (faster downloads)

Dataset loaded successfully!
Total number of samples: 209261

Column names: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set']


## Sample Random Entries

Filtering the dataset to English samples only, then selecting 5 random samples with a fixed seed for reproducibility.

In [5]:
# Set random seed for reproducibility
random.seed(42)

# Set number of samples
number_samples = 2

# Filter dataset to English only
print("Filtering dataset to English samples only...")
english_data = train_data.filter(lambda x: x['language'] == 'en')
print(f"English samples: {len(english_data)} out of {len(train_data)} total")

# Sample random indices from English data
sample_indices = random.sample(range(len(english_data)), number_samples)
print(f"\nSelected indices from English dataset: {sample_indices}")

# Get the samples
samples = [english_data[i] for i in sample_indices]
print(f"Successfully sampled {len(samples)} English entries")

# Verify all samples are English
languages = [s['language'] for s in samples]
print(f"Languages: {languages}")
assert all(lang == 'en' for lang in languages), "Not all samples are English!"

Filtering dataset to English samples only...
English samples: 43501 out of 209261 total

Selected indices from English dataset: [41905, 7296]
Successfully sampled 2 English entries
Languages: ['en', 'en']


## Examine the Samples

Let's look at each sample to understand the data structure.

In [6]:
for i, sample in enumerate(samples):
    print(f"\n{'='*80}")
    print(f"SAMPLE {i} (ID: {sample['id']}, Language: {sample['language']})")
    print(f"{'='*80}")
    
    print(f"\n📄 SOURCE TEXT (with PII):")
    print(f"{sample['source_text']}")
    
    print(f"\n🔒 TARGET TEXT (masked):")
    print(f"{sample['target_text']}")
    
    print(f"\n🏷️  GROUND TRUTH PII:")
    for pii_item in sample['privacy_mask']:
        print(f"  - {pii_item['label']:20s} : '{pii_item['value']}' (pos {pii_item['start']}-{pii_item['end']})")


SAMPLE 0 (ID: 207666, Language: en)

📄 SOURCE TEXT (with PII):
Analyst, we need you to investigate further about the Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12.2.4 case. Draw insights from this situation and propose how it can affect our company policies.

🔒 TARGET TEXT (masked):
[JOBTYPE], we need you to investigate further about the [USERAGENT] case. Draw insights from this situation and propose how it can affect our company policies.

🏷️  GROUND TRUTH PII:
  - JOBTYPE              : 'Analyst' (pos 0-7)
  - USERAGENT            : 'Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12.2.4' (pos 54-128)

SAMPLE 1 (ID: 173057, Language: en)

📄 SOURCE TEXT (with PII):
Our main blockchain consultant identified 13eBdNVhaUZ6RPaCbiD1jPWVnEKaRCUU and 0xffff97e7cbbdc819da9d6ae8ef80deaf4bf39cec as two significant cryptocurrency addresses to monitor considering their possible impact on our business continuity.

🔒 TARGET TEXT (masked):
Our main blockc

## Setup OpenAI API

Configure the OpenAI client using the API key from environment variables.

In [7]:
# Get API key from environment variable
api_key = os.getenv("OPENAI_API_KEY")

# Validate API key
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.\n\n")

# Check if it's still the placeholder value from .env.example
if api_key.startswith("your-") or "placeholder" in api_key.lower() or "example" in api_key.lower():
    raise ValueError("OPENAI_API_KEY appears to be the placeholder value from .env.example.\n\n")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)
print("✓ OpenAI client initialized successfully")

✓ OpenAI client initialized successfully


## Send Samples to OpenAI for PII Detection

Process each sample and get PII detection results from OpenAI.

In [33]:
from collections.abc import Callable

def detect_pii_with_model(sample, model_name, client, get_prompt: Callable[[str], str]):
    """
    Detect PII in a sample using a specific OpenAI model.
    
    Args:
        sample: Dataset sample containing 'source_text'
        model_name: OpenAI model name (e.g., 'gpt-4o-mini', 'gpt-5.2')
        client: OpenAI client instance
        get_prompt: function that generates prompt
    Returns:
        dict: Result containing detected PII or error information
    """

    prompt = get_prompt(sample['source_text'])

    result = {
            "sample_id": sample['id'],
            "source_text": sample['source_text'],
            "ground_truth": sample['privacy_mask'],
            "model": model_name
    }

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a PII detection expert. Return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # deterministic output
            response_format={"type": "json_object"}
        )
        
        # Extract the response
        llm_response = response.choices[0].message.content
        
        detected_pii = json.loads(llm_response)

        result["detected_pii"] = detected_pii
        result["raw_response"] = llm_response

        return result
        
    except json.JSONDecodeError as e:
        result["error"] = "Invalid JSON response"
        return result
    except Exception as e:
        result["error"] = str(e)
        return result


## Send Samples to OpenAI for PII Detection

Process each sample and get PII detection results from two OpenAI models:
- **gpt-4o-mini**: Cost-effective baseline model
- **gpt-5.2**: Latest and most capable model (as of Jan 2026)

In [35]:
models_to_test = [
    "gpt-4o-mini",  # Cost-effective baseline
    "gpt-5.2"       # Latest flagship model
]

def print_result(result):
    if 'error' in result:
        print(f"   ❌ Error: {result['error']}")
        print(f"   ✓ Raw Response: {result['raw_response'].strip().replace('\n', '')}")
    else:
        print(f"   ✓ Response: {", ".join(result["detected_pii"]["pii"])}")

def print_sample_data(sample):
    # Show sample text
    print(f"\n📄 Text: {sample['source_text']}")

    # Show expected PII (ground truth)
    print(f"\n🏷️  Expected PII ({len(sample['privacy_mask'])} items):")
    for pii_item in sample['privacy_mask']:
        print(f"   • {pii_item['label']:20s} : '{pii_item['value']}'")

def print_header(sample):
    print(f"\n{'='*80}")
    print(f"Processing Sample {i} (ID: {sample['id']})")
    print(f"{'='*80}")

def test_prompt_with_models(samples, get_prompt: Callable[[str], str]):
    results_for_prompt_per_model: dict[str, list[object]] = {}
    for i, sample in enumerate(samples):
        # Test with each model
        for model_name in models_to_test:
            result = detect_pii_with_model(sample, model_name, client, get_prompt)
            results_for_prompt_per_model.setdefault(model_name, []).append(result)
    return results_for_prompt_per_model

def print_results_for_prompt(results_for_prompt_per_model: dict[str, list[object]], samples, num_samples_to_print):
    for i in range(num_samples_to_print):
        print_header(samples[i])
        print_sample_data(samples[i])

        for model_name in models_to_test:
            print(f"\n🤖 Testing with {model_name}...")
            print_result(results_for_prompt_per_model[model_name][i])

## Define Simple PII Detection Prompt

Create a prompt that asks the LLM to identify all PII in the text.

In [38]:
def gety_pii_prompt_simplest(text):
    """Create a prompt for PII detection."""
    return f"""For the text provided below identify all personally identifiable information (PII) in it.
               Return your response as a JSON array that contains all the portions of text identified as PII.
               Only return the JSON, no additional text. JSON array should be named pii, for example {{'pii': ['value1', 'value']}}.
               Text to analyze: {text}
               """


## Call simple prompt

In [39]:
# process all the samples
results_for_prompt_per_model = test_prompt_with_models(samples, gety_pii_prompt_simplest)

In [40]:
# print first N results
print_results_for_prompt(results_for_prompt_per_model, samples, 2)


Processing Sample 1 (ID: 207666)

📄 Text: Analyst, we need you to investigate further about the Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12.2.4 case. Draw insights from this situation and propose how it can affect our company policies.

🏷️  Expected PII (2 items):
   • JOBTYPE              : 'Analyst'
   • USERAGENT            : 'Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12.2.4'

🤖 Testing with gpt-4o-mini...
   ✓ Response: 

🤖 Testing with gpt-5.2...
   ✓ Response: 

Processing Sample 1 (ID: 173057)

📄 Text: Our main blockchain consultant identified 13eBdNVhaUZ6RPaCbiD1jPWVnEKaRCUU and 0xffff97e7cbbdc819da9d6ae8ef80deaf4bf39cec as two significant cryptocurrency addresses to monitor considering their possible impact on our business continuity.

🏷️  Expected PII (2 items):
   • BITCOINADDRESS       : '13eBdNVhaUZ6RPaCbiD1jPWVnEKaRCUU'
   • ETHEREUMADDRESS      : '0xffff97e7cbbdc819da9d6ae8ef80deaf4bf39cec'

🤖 Testing with gpt-4o-mini

## Calculate the stats

In [41]:
def calculate_stats_for_model(model_results):
    """
    Calculate PII detection statistics for a single model.
    
    Args:
        model_results: List of result objects from detect_pii_with_model
        
    Returns:
        dict: Statistics including missed count, extra count, and details
    """
    total_ground_truth = 0
    total_detected = 0
    total_missed = 0
    total_extra = 0
    sample_details = []
    
    for result in model_results:
        if 'error' in result:
            # Skip samples with errors
            continue
            
        # Get ground truth values
        ground_truth_values = set(item['value'] for item in result['ground_truth'])
        
        # Get detected values
        detected_values = set(result.get('detected_pii', {}).get('pii', []))
        
        # Calculate missed (in ground truth but not detected)
        missed = ground_truth_values - detected_values
        
        # Calculate extra (detected but not in ground truth)
        extra = detected_values - ground_truth_values
        
        # Calculate correct (intersection)
        correct = ground_truth_values & detected_values
        
        total_ground_truth += len(ground_truth_values)
        total_detected += len(detected_values)
        total_missed += len(missed)
        total_extra += len(extra)
        
        sample_details.append({
            'sample_id': result['sample_id'],
            'ground_truth_count': len(ground_truth_values),
            'detected_count': len(detected_values),
            'correct_count': len(correct),
            'missed_count': len(missed),
            'extra_count': len(extra),
            'missed_values': list(missed),
            'extra_values': list(extra)
        })
    
    # Calculate rates
    recall = (total_ground_truth - total_missed) / total_ground_truth if total_ground_truth > 0 else 0
    precision = (total_detected - total_extra) / total_detected if total_detected > 0 else 0
    
    return {
        'total_ground_truth': total_ground_truth,
        'total_detected': total_detected,
        'total_missed': total_missed,
        'total_extra': total_extra,
        'recall': recall,
        'precision': precision,
        'sample_details': sample_details
    }

In [42]:
def calculate_stats_for_all_models(results_per_model):
    """
    Calculate PII detection statistics for all models.
    
    Args:
        results_per_model: Dict mapping model name to list of result objects
        
    Returns:
        dict: Statistics per model
    """
    all_stats = {}
    
    for model_name, model_results in results_per_model.items():
        stats = calculate_stats_for_model(model_results)
        all_stats[model_name] = stats
    
    return all_stats


def print_stats(all_stats):
    """Print statistics in a formatted way."""
    for model_name, stats in all_stats.items():
        print(f"\n{'='*60}")
        print(f"Statistics for {model_name}")
        print(f"{'='*60}")
        print(f"Total ground truth PII: {stats['total_ground_truth']}")
        print(f"Total detected PII:     {stats['total_detected']}")
        print(f"Total missed:           {stats['total_missed']}")
        print(f"Total extra (false +):  {stats['total_extra']}")
        print(f"Recall:                 {stats['recall']:.2%}")
        print(f"Precision:              {stats['precision']:.2%}")
        
        print(f"\nPer-sample breakdown:")
        for detail in stats['sample_details']:
            print(f"  Sample {detail['sample_id']}:")
            print(f"    Ground truth: {detail['ground_truth_count']}, Detected: {detail['detected_count']}")
            print(f"    Correct: {detail['correct_count']}, Missed: {detail['missed_count']}, Extra: {detail['extra_count']}")
            if detail['missed_values']:
                print(f"    Missed values: {detail['missed_values']}")
            if detail['extra_values']:
                print(f"    Extra values: {detail['extra_values']}")

In [43]:
# Calculate and print statistics for all models
all_stats = calculate_stats_for_all_models(results_for_prompt_per_model)
print_stats(all_stats)


Statistics for gpt-4o-mini
Total ground truth PII: 4
Total detected PII:     0
Total missed:           4
Total extra (false +):  0
Recall:                 0.00%
Precision:              0.00%

Per-sample breakdown:
  Sample 207666:
    Ground truth: 2, Detected: 0
    Correct: 0, Missed: 2, Extra: 0
    Missed values: ['Analyst', 'Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12.2.4']
  Sample 173057:
    Ground truth: 2, Detected: 0
    Correct: 0, Missed: 2, Extra: 0
    Missed values: ['0xffff97e7cbbdc819da9d6ae8ef80deaf4bf39cec', '13eBdNVhaUZ6RPaCbiD1jPWVnEKaRCUU']

Statistics for gpt-5.2
Total ground truth PII: 4
Total detected PII:     2
Total missed:           2
Total extra (false +):  0
Recall:                 50.00%
Precision:              100.00%

Per-sample breakdown:
  Sample 207666:
    Ground truth: 2, Detected: 0
    Correct: 0, Missed: 2, Extra: 0
    Missed values: ['Analyst', 'Mozilla/5.0 (Windows NT 5.1; WOW64; rv:12.2) Gecko/20100101 Firefox/12